## Task 3: Fine Tune NER Model
Objective: Fine-tune a Named Entity Recognition (NER) model to extract key entities (e.g., products, prices, and location) from Amharic Telegram messages.
Steps:
Use Google Colab or any other environment with GPU support for faster training.
Install necessary libraries by running the following commands:
You will use the pre-trained XLM-Roberta or bert-tiny-amharic or afroxmlr model, which supports multilingual tasks, including Amharic.
Load the labeled dataset in CoNLL format from the previous task.
You can use Hugging Face's datasets library to load the data or manually parse the CoNLL format into a pandas DataFrame.
Tokenize the data and align the labels with tokens produced by the tokenizer
Set up training arguments, such as learning rate, number of epochs, batch size, and evaluation strategy.



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Install Hugging Face Transformers, Datasets, and PEFT (for parameter-efficient fine-tuning)
!pip install transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8311b3a35a409d50c8aa3c4f03d7dd4c81fe41d4ddfa5eff6367c9343c995c8e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [13]:

import pandas as pd
from datasets import load_dataset, Dataset, ClassLabel
from transformers import XLMRobertaTokenizer, Trainer, TrainingArguments, XLMRobertaForTokenClassification
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [ ]:

def load_conll_dataset(file_path):
    sentences, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence, label = [], []
        for line in f:
            line = line.strip()
            if line:
                token, tag = line.split()
                sentence.append(token)
                label.append(tag)
            else:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
        if sentence:
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

# Load your CoNLL dataset
file_path = r'/content/drive/MyDrive/labele_data.conll'
sentences, labels = load_conll_dataset(file_path)

# Create a DataFrame
data = {'tokens': sentences, 'ner_tags': labels}
df = pd.DataFrame(data)

# Convert labels to ClassLabel
unique_labels = set(tag for label in labels for tag in label)
class_labels = ClassLabel(names=list(unique_labels))

# Create a Dataset object
dataset = Dataset.from_pandas(df)

# Step 3: Tokenize the data
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if id is None else class_labels.str2int(label[id]) for id in word_ids]
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Step 4: Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Step 5: Fine-tune the model
#num_labels = len(class_labels)
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Use a separate validation set if available
    tokenizer=tokenizer,
)

trainer.train()

# Step 6: Evaluate the model
predictions, labels, _ = trainer.predict(tokenized_dataset)
predictions = np.argmax(predictions, axis=2)

# Convert predictions and labels to a list format
true_labels = [[class_labels.int2str(label) for label in label_row if label != -100] for label_row in labels]
pred_labels = [[class_labels.int2str(pred) for pred, label in zip(pred_row, label_row) if label != -100] for pred_row, label_row in zip(predictions, labels)]

# Calculate precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')

print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

# Step 7: Save the model
model.save_pretrained('./fine_tuned_ner_model')
tokenizer.save_pretrained('./fine_tuned_ner_model')

print("Model saved successfully!")

Map:   0%|          | 0/3257 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-17-e9c93695efc5>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: